# Spark Count Vectorizer

## References

https://spark.apache.org/docs/latest/ml-features.html#countvectorizer

https://github.com/apache/spark/blob/master/examples/src/main/python/ml/count_vectorizer_example.py

https://spark.apache.org/docs/2.2.0/api/python/pyspark.ml.html#pyspark.ml.feature.CountVectorizerModel

https://stackoverflow.com/questions/36349281/how-to-loop-through-each-row-of-dataframe-in-pyspark

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import CountVectorizer

In [2]:
spark = SparkSession\
        .builder\
        .appName("CountVectorizerExample")\
        .getOrCreate()

In [3]:
df = spark.createDataFrame([
        (0, "a b c".split(" ")),
        (1, "a b b c a".split(" ")),
        (1, "a b b e a".split(" ")),
        (1, "a b b c e".split(" ")),
        (1, "a b d c a".split(" ")),
        (1, "e e e e e e e e e e".split(" ")),
        (0, "a f f g a f".split(" "))
    ], ["id", "words"])

In [4]:
df.show()

+---+--------------------+
| id|               words|
+---+--------------------+
|  0|           [a, b, c]|
|  1|     [a, b, b, c, a]|
|  1|     [a, b, b, e, a]|
|  1|     [a, b, b, c, e]|
|  1|     [a, b, d, c, a]|
|  1|[e, e, e, e, e, e...|
|  0|  [a, f, f, g, a, f]|
+---+--------------------+



In [5]:
# fit a CountVectorizerModel from the corpus.
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=4, minDF=2.0)

In [7]:
model = cv.fit(df)

In [8]:
print('Min TF', model.getMinTF())
print('Min DF', model.getMinDF())
print('Max DF', model.getMaxDF())

Min TF 1.0
Min DF 2.0
Max DF 9.223372036854776e+18


In [9]:
model.vocabulary

['e', 'a', 'b', 'c']

It appears that the default order of model.vocabulary is the order of significance of the features in descending order. Notice that 'e' appears first and it is most represented in the DataFrame.

In [10]:
result = model.transform(df)

In [11]:
result.show(truncate=False)

+---+------------------------------+-------------------------------+
|id |words                         |features                       |
+---+------------------------------+-------------------------------+
|0  |[a, b, c]                     |(4,[1,2,3],[1.0,1.0,1.0])      |
|1  |[a, b, b, c, a]               |(4,[1,2,3],[2.0,2.0,1.0])      |
|1  |[a, b, b, e, a]               |(4,[0,1,2],[1.0,2.0,2.0])      |
|1  |[a, b, b, c, e]               |(4,[0,1,2,3],[1.0,1.0,2.0,1.0])|
|1  |[a, b, d, c, a]               |(4,[1,2,3],[2.0,1.0,1.0])      |
|1  |[e, e, e, e, e, e, e, e, e, e]|(4,[0],[10.0])                 |
|0  |[a, f, f, g, a, f]            |(4,[1],[2.0])                  |
+---+------------------------------+-------------------------------+



It appears that indexes in the features vector refer to indices in the `model.vocabulary` list. Note that model.vocabulary[0] is 'e'. Counts of [0] are equal to counts of 'e'.

In [12]:
feature_counts_dictionary = dict()
for row in result.rdd.collect():
    #print(row)
    #print(row.features.indices)
    #print(row.features)
    #print(row.features[1])
    for i in range(0,len(row.features.indices)):
        feature_id = row.features.indices[i]
        
        feature_name = model.vocabulary[feature_id]
        #print(feature_name)
        feature_count = row.features[int(feature_id)]
        #print(feature_count)
        #print("feature_id {0}, feature_count {1}".format(row.features.indices[i], row.features[i]))
        if feature_name in feature_counts_dictionary:
            feature_counts_dictionary[feature_name] += feature_count
        else:
            feature_counts_dictionary[feature_name] =  feature_count
        

In [13]:
print('Most Informative Features')
for feature in model.vocabulary:
    print(feature, feature_counts_dictionary[feature])

Most Informative Features
e 12.0
a 10.0
b 8.0
c 4.0
